# Aggregation Mini Challenge 2

Apply what you have learned so far about the aggregation framework to try solve this mini challenge

In [1]:
# Import pymongo dependencies
from pymongo import MongoClient
client = MongoClient(host="localhost",port=27017)

In [3]:
# Set up your MongoClient and database variable
db = client['aggregation_test']


['products', 'users', 'beauty_products', 'orders']

In [4]:
# Function which prints out all the results of a cursor
def print_cursor(cursor):
    for document in cursor:
        print(document, end="\n\n")

In [111]:
# Challenge Part 1:
# Find the top sellers (by user name) based on the quantity of all prodcuts they have sold from the 'orders' collection.
# Sort the sellers from highest to lowest quantity sold, and output the results to a new collection called 'top_sellers'
###
# Example entry from new collection:
# {"_id": ObjectId(...), "seller_name": "Sarah", "num_items_sold": 7}

challenge1_cursor = db.orders.aggregate([
    {"$unwind": "$items"},
    {"$group":
     {
         "_id": "$items.product_id",
         "num_items_sold": {"$sum": "$items.quantity"}
     }
     },
    {
        "$lookup":
        {
            "from": "products",
            "localField": "_id",
            "foreignField": "_id",
            "as": "products"
        },

    },
    {
        "$group":
        {
            "_id": {"$first": "$products.seller_id"},
            "num_items_sold": {"$sum": "$num_items_sold"}
        }
    },
    {
        "$lookup":
        {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "seller"
        },

    },
    {"$project": {"seller_name": {"$first":"$seller.name"}, "num_items_sold": 1}},
    {"$sort": {"num_items_sold": -1}}
])

In [112]:
# Grab the results from your database
print_cursor(challenge1_cursor)

{'_id': ObjectId('66fbbbd37bad260de02d2210'), 'num_items_sold': 7, 'seller_name': 'Sarah'}

{'_id': ObjectId('66fbbbd37bad260de02d2211'), 'num_items_sold': 5, 'seller_name': 'Bob'}

{'_id': ObjectId('66fbbbd37bad260de02d2214'), 'num_items_sold': 2, 'seller_name': 'Jessica'}

{'_id': ObjectId('66fbbbd37bad260de02d2213'), 'num_items_sold': 1, 'seller_name': 'Lisa'}



In [148]:
### Challenge Part 2:
###   You want to reward your sellers with an extra day of a premium membership for each item they've sold. Add
###   an attribute called 'premium_exp_date' to the entries in the 'users' collection which contains the date their 
###   premium membership will expire. Currently no users have a premium membership, so use today as the start date
###   from their premium membership, but make sure your pipeline is able to add the extra days of premium to an
###   existing date if one existed. 
###
### Example entry from users collection: 
###   {'_id': ObjectId(...), 'name': 'Sarah', 'premium_exp_date': datetime.datetime(2021, 12, 24, 4, 2, 4, 146000)}

challenge2_cursor = db.orders.aggregate([
    {"$unwind": "$items"},
    {"$group":
     {
         "_id": "$items.product_id",
         "num_items_sold": {"$sum": "$items.quantity"}
     }
     },
    {
        "$lookup":
        {
            "from": "products",
            "localField": "_id",
            "foreignField": "_id",
            "as": "products"
        },

    },
    {
        "$group":
        {
            "_id": {"$first": "$products.seller_id"},
            "num_items_sold": {"$sum": "$num_items_sold"}
        }
    },
    {
        "$lookup":
        {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "seller"
        },

    },
    {"$project": {"seller_name": {"$first":"$seller.name"}, "num_items_sold": 1, 
                  "premium_exp_date": {
                      "$switch":{"branches":[
                          {"case": {"$eq":[{"$first":"$seller.premium_exp_date"}, None]},
                           "then":{"$dateAdd":{"startDate": "$$NOW", "unit":"day", "amount": "$num_items_sold"}}
                           },
                          {"case": {"$lte":[{"$first":"$seller.premium_exp_date"}, "$$NOW"]},
                           "then":{"$dateAdd":{"startDate": "$$NOW", "unit":"day", "amount": "$num_items_sold"}}
                           },
                          {"case": {"$gt":[{"$first":"$seller.premium_exp_date"}, "$$NOW"]},
                           "then":{"$dateAdd":{"startDate": {"$first":"$seller.premium_exp_date"}, "unit":"day", "amount": "$num_items_sold"}}
                           }
                      ]}

    }}},
    {"$sort": {"num_items_sold": -1}}
])

In [149]:
# Print out your results
print_cursor(challenge2_cursor)

{'_id': ObjectId('66fbbbd37bad260de02d2210'), 'num_items_sold': 7, 'seller_name': 'Sarah', 'premium_exp_date': datetime.datetime(2024, 10, 10, 16, 28, 20, 778000)}

{'_id': ObjectId('66fbbbd37bad260de02d2211'), 'num_items_sold': 5, 'seller_name': 'Bob', 'premium_exp_date': datetime.datetime(2024, 11, 5, 0, 0)}

{'_id': ObjectId('66fbbbd37bad260de02d2214'), 'num_items_sold': 2, 'seller_name': 'Jessica', 'premium_exp_date': datetime.datetime(2024, 10, 5, 16, 28, 20, 778000)}

{'_id': ObjectId('66fbbbd37bad260de02d2213'), 'num_items_sold': 1, 'seller_name': 'Lisa', 'premium_exp_date': datetime.datetime(2024, 10, 4, 16, 28, 20, 778000)}

